### 내 사전으로 문장 라벨링

In [3]:
import pandas as pd
from ekonlpy.tag import Mecab
import ast
import logging

# 로깅 설정 (디버깅용)
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# ekonlpy의 Mecab 토크나이저 객체 생성
mecab = Mecab()

# 1. 감정 사전 로드 및 전처리
try:
    dovish_dict = pd.read_csv('final_dovish.csv')  # 비둘기파 사전
    hawkish_dict = pd.read_csv('final_hawkish.csv')  # 매파 사전
except FileNotFoundError as e:
    print(f"파일 로드 오류: {e}")
    exit()

# 결측값 처리 및 단어 리스트 생성 (컬럼명 'Dovish', 'Hawkish' 가정)
dovish_dict = dovish_dict.dropna(subset=['Dovish'])
hawkish_dict = hawkish_dict.dropna(subset=['Hawkish'])

dovish_words = set(dovish_dict['Dovish'].str.strip())
hawkish_words = set(hawkish_dict['Hawkish'].str.strip())

# 감정 사전 내용 출력 (디버깅용)
logger.info(f"Dovish words (sample): {list(dovish_words)[:10]}")
logger.info(f"Hawkish words (sample): {list(hawkish_words)[:10]}")

# 2. 문장 데이터 로드
try:
    sentences_df = pd.read_csv('conferences_sentences.csv')  # 문장 데이터
except FileNotFoundError as e:
    print(f"파일 로드 오류: {e}")
    exit()

# 결측값 처리
sentences_df['sentences'] = sentences_df['sentences'].fillna('[]')

# 3. 리스트 형식의 문자열을 실제 리스트로 변환
def parse_sentences(sentences_str):
    try:
        # 문자열로 저장된 리스트를 실제 리스트로 변환
        sentences_list = ast.literal_eval(sentences_str)
        return sentences_list if isinstance(sentences_list, list) else [sentences_str]
    except (ValueError, SyntaxError):
        # 파싱 실패 시 원래 문자열을 리스트로 처리
        return [sentences_str]

# 문장 리스트를 개별 문장으로 분리
sentences_expanded = []
for idx, row in sentences_df.iterrows():
    sentences_list = parse_sentences(row['sentences'])
    for sentence in sentences_list:
        sentences_expanded.append({'sentence': sentence})

# 새로운 데이터프레임 생성
expanded_df = pd.DataFrame(sentences_expanded)

# 결측값 처리
expanded_df['sentence'] = expanded_df['sentence'].fillna('')

# 4. 문장을 라벨링하는 함수 정의 (디버깅 정보 포함)
def label_sentence(sentence):
    # 문장에서 단어 추출 (ekonlpy의 Mecab으로 토큰화)
    tokens = mecab.morphs(sentence)
    
    # 매파/비둘기파 단어 매칭
    dovish_matched = [token for token in tokens if token in dovish_words]
    hawkish_matched = [token for token in tokens if token in hawkish_words]
    
    # 매칭된 단어 카운트
    dovish_count = len(dovish_matched)
    hawkish_count = len(hawkish_matched)
    
    # 라벨링 기준
    if dovish_count > hawkish_count:
        label = 'Dovish'
    elif hawkish_count > dovish_count:
        label = 'Hawkish'
    else:
        label = 'Neutral'
    
    # 디버깅 정보 출력
    logger.info(f"Sentence: {sentence}")
    logger.info(f"Dovish matches: {dovish_matched}, Count: {dovish_count}")
    logger.info(f"Hawkish matches: {hawkish_matched}, Count: {hawkish_count}")
    logger.info(f"Label: {label}\n")
    
    return label

# 5. 각 문장에 대해 라벨링 수행
expanded_df['Label'] = expanded_df['sentence'].apply(label_sentence)

# 6. 결과를 CSV 파일로 저장
expanded_df.to_csv('labeled_sentences.csv', index=False, encoding='utf-8-sig')

print("라벨링 완료! 결과는 'labeled_sentences.csv' 파일에 저장되었습니다.")

Dovish words (sample): ['비메모리', '장터', '양질', '조달', '건설기업', '평화주의', '급평가', '디플레션갭', '생산력', '수소']
Hawkish words (sample): ['부채한도', '김건희', '외부요인', '기술기업', '장남', '현직', '라더', '사인', '탈퇴', '폭력']
Sentence: 금융통화위원회는 다음 통화정책방향 결정시까지 한국은행 기준금리를 현 수준(2.00%)에서 유지하여 통화정책을 운용하기로 하였음
Dovish matches: ['유지', '운용'], Count: 2
Hawkish matches: ['기준금리'], Count: 1
Label: Dovish

Sentence: 최근 국내 경기는 내수와 수출 모두 감소세를 지속하면서계속 위축되고 있으나 하강속도는 다소 완만해지고 있으며 세계경제 침체, 국제금융시장 불안 등으로 인해 향후성장의 하향위험은 여전히 큰 것으로 판단됨
Dovish matches: ['내수', '수출', '감소세', '지속', '위축', '하강', '완만', '해지', '침체', '국제', '성장', '하향', '위험'], Count: 13
Hawkish matches: ['불안', '성장', '위험'], Count: 3
Label: Dovish

Sentence: 소비자물가는 국제유가가 안정세를 보이는 가운데 경기부진에 따른 수요압력 완화로 오름세가 둔화되었으며 앞으로도 이러한 추세가 이어질 것으로 보임.
Dovish matches: ['국제', '유가', '안정세', '부진', '수요', '완화', '둔화', '추세', '보임'], Count: 9
Hawkish matches: ['압력'], Count: 1
Label: Dovish

Sentence: 부동산시장에서는 거래위축 및 가격하락 현상이 지속되고 있음
Dovish matches: ['위축', '하락', '지속'], Count: 3
Hawkish matches: ['거래'], Count: 1
Label: 

라벨링 완료! 결과는 'labeled_sentences.csv' 파일에 저장되었습니다.


### 나이브베이즈 분류기로 학습 및 평균 정확도 계산

In [1]:
import pandas as pd
import numpy as np
from ekonlpy.tag import Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import logging

# 로깅 설정 (디버깅용)
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# ekonlpy의 Mecab 토크나이저 객체 생성
mecab = Mecab()

# 1. 데이터 로드
try:
    df = pd.read_csv('문장라벨.csv')  # 수작업으로 라벨링된 데이터
except FileNotFoundError as e:
    print(f"파일 로드 오류: {e}")
    exit()

# 결측값 처리
df['sentence'] = df['sentence'].fillna('')
df['Label'] = df['Label'].fillna('Neutral')

# 2. Hawkish와 Dovish 문장만 필터링 (이진 분류)
binary_df = df[df['Label'].isin(['Hawkish', 'Dovish'])].copy()
logger.info(f"Hawkish와 Dovish 문장 수: {len(binary_df)}")

# 3. 토큰화 함수 정의
def tokenize(text):
    return mecab.morphs(text)

# 4. 나이브 베이즈 분류 실험
n_iterations = 30
accuracies = []

# TF-IDF 벡터라이저 설정
vectorizer = TfidfVectorizer(tokenizer=tokenize, lowercase=False)

# 30회 반복 실험
for i in range(n_iterations):
    # 60% 학습 데이터, 40% 테스트 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(
        binary_df['sentence'], binary_df['Label'], 
        test_size=0.4, random_state=i, stratify=binary_df['Label']
    )

    # TF-IDF 벡터화
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    # 나이브 베이즈 분류기 학습
    nb_classifier = MultinomialNB()
    nb_classifier.fit(X_train_tfidf, y_train)

    # 테스트 데이터로 예측
    y_pred = nb_classifier.predict(X_test_tfidf)

    # 정확도 계산
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    logger.info(f"Iteration {i+1}: Accuracy = {accuracy:.4f}")

# 5. 평균 정확도 계산
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
logger.info(f"\n평균 정확도: {mean_accuracy:.4f}")
logger.info(f"표준편차: {std_accuracy:.4f}")

# 6. 결과 평가
if mean_accuracy >= 0.86:
    logger.info("분류기의 평균 정확도가 약 86%로 기준 이상(above par)입니다.")
else:
    logger.info("분류기의 평균 정확도가 기준(86%)에 미치지 못합니다.")

# 7. 결과 저장 (필요 시)
results_df = pd.DataFrame({
    'Iteration': range(1, n_iterations+1),
    'Accuracy': accuracies
})
results_df.to_csv('재실험.csv', index=False, encoding='utf-8-sig')
logger.info("실험 결과가 'nbc_classification_results.csv' 파일에 저장되었습니다.")

Hawkish와 Dovish 문장 수: 2239
c:\Users\yoons\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Iteration 1: Accuracy = 0.9051
c:\Users\yoons\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Iteration 2: Accuracy = 0.8996
c:\Users\yoons\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Iteration 3: Accuracy = 0.8984
c:\Users\yoons\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Iteration 4: Accuracy = 0.8973
c:\Users\yoons\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: Us

### 말이 안되는 점수인데? 오류가 난게 분명하다

라벨링한 데이터를 갖고 하나하나 살펴보며, 수작업으로 라벨링을 진행

In [2]:
import pandas as pd
from ekonlpy.tag import Mecab
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import logging

# 로깅 설정 (디버깅용)
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# ekonlpy의 Mecab 토크나이저 객체 생성
mecab = Mecab()

# 1. 감정 사전 로드 및 전처리
try:
    dovish_dict = pd.read_csv('final_dovish.csv')
    hawkish_dict = pd.read_csv('final_hawkish.csv')
except FileNotFoundError as e:
    print(f"파일 로드 오류: {e}")
    exit()

dovish_dict = dovish_dict.dropna(subset=['Dovish'])
hawkish_dict = hawkish_dict.dropna(subset=['Hawkish'])

dovish_words = set(dovish_dict['Dovish'].str.strip())
hawkish_words = set(hawkish_dict['Hawkish'].str.strip())

logger.info(f"Dovish words (sample): {list(dovish_words)[:10]}")
logger.info(f"Hawkish words (sample): {list(hawkish_words)[:10]}")

# 2. 테스트 데이터 로드
try:
    test_df = pd.read_csv('문장라벨.csv')
except FileNotFoundError:
    print("새로운 테스트 데이터 파일이 없습니다. 기존 데이터에서 추출합니다.")
    labeled_df = pd.read_csv('labeled_sentences.csv')
    test_df = labeled_df.sample(frac=0.2, random_state=42)
    test_df = test_df[test_df['Label'].isin(['Hawkish', 'Dovish'])].copy()

test_df['sentence'] = test_df['sentence'].fillna('')
test_df['Label'] = test_df['Label'].fillna('Neutral')

test_df = test_df[test_df['Label'].isin(['Hawkish', 'Dovish'])].copy()
logger.info(f"테스트 데이터 크기: {len(test_df)} 문장")
logger.info(f"테스트 데이터 클래스 분포:\n{test_df['Label'].value_counts()}")

# 3. 문장을 라벨링하는 함수 정의 (디버깅 포함)
def label_sentence(sentence):
    tokens = mecab.morphs(sentence)
    dovish_matched = [t for t in tokens if t in dovish_words]
    hawkish_matched = [t for t in tokens if t in hawkish_words]
    dovish_count = len(dovish_matched)
    hawkish_count = len(hawkish_matched)
    logger.info(f"Sentence: {sentence}")
    logger.info(f"Dovish matches: {dovish_matched}, Count: {dovish_count}")
    logger.info(f"Hawkish matches: {hawkish_matched}, Count: {hawkish_count}")
    if hawkish_count > dovish_count:
        label = 'Hawkish'
    elif dovish_count > hawkish_count:
        label = 'Dovish'
    else:
        label = 'Neutral'
    logger.info(f"Predicted: {label}\n")
    return label

# 4. 예측 수행
test_df['Predicted_Label'] = test_df['sentence'].apply(label_sentence)

# 중립 제외 로직 제거 (필요 시 주석 해제)
# test_df = test_df[test_df['Predicted_Label'].isin(['Hawkish', 'Dovish'])].copy()
logger.info(f"최종 예측 데이터 크기: {len(test_df)} 문장")
logger.info(f"예측 라벨 클래스 분포:\n{test_df['Predicted_Label'].value_counts()}")

# 5. 평가 지표 계산
true_labels = test_df['Label'].values
pred_labels = test_df['Predicted_Label'].values

# 예측과 실제 라벨 비교 (일부만 출력)
for idx, (true, pred) in enumerate(zip(true_labels[:10], pred_labels[:10])):
    logger.info(f"Sample {idx+1}: True={true}, Predicted={pred}")

# 중립 예측이 포함된 경우 제외 (필요 시 조정)
eval_df = test_df[test_df['Predicted_Label'].isin(['Hawkish', 'Dovish'])].copy()
if len(eval_df) == 0:
    logger.info("평가할 데이터가 없습니다. 모든 예측이 중립(Neutral)입니다.")
    exit()

true_labels = eval_df['Label'].values
pred_labels = eval_df['Predicted_Label'].values

accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels, pos_label='Hawkish')
recall = recall_score(true_labels, pred_labels, pos_label='Hawkish')
f1 = f1_score(true_labels, pred_labels, pos_label='Hawkish')

cm = confusion_matrix(true_labels, pred_labels, labels=['Hawkish', 'Dovish'])
tn, fp, fn, tp = cm.ravel()

# 6. 결과 출력
logger.info("\n=== 평가 결과 ===")
logger.info(f"True Positives (TP): {tp}")
logger.info(f"True Negatives (TN): {tn}")
logger.info(f"False Positives (FP): {fp}")
logger.info(f"False Negatives (FN): {fn}")
logger.info(f"정확도 (Accuracy): {accuracy:.4f}")
logger.info(f"정밀도 (Precision): {precision:.4f}")
logger.info(f"재현율 (Recall): {recall:.4f}")
logger.info(f"F1 점수 (F1 Score): {f1:.4f}")

# 7. 결과 저장
eval_df.to_csv('test_predictions_with_metrics.csv', index=False, encoding='utf-8-sig')
logger.info("예측 결과 및 평가 지표가 'test_predictions_with_metrics.csv' 파일에 저장되었습니다.")

Dovish words (sample): ['올리브영', '중국건설은행', '자체사업', '국제화', '웰빙', '궤도', '경영실태평가', '청정에너지', '정부자금', '하나다']
Hawkish words (sample): ['중요도', '노도', '선뜻', '불편', '손발', '튤립', '출입', '출발', '결정자', '증상']
테스트 데이터 크기: 2239 문장
테스트 데이터 클래스 분포:
Label
Dovish     1913
Hawkish     326
Name: count, dtype: int64
Sentence: 금융통화위원회는 다음 통화정책방향 결정시까지 한국은행 기준금리를 현 수준(2.00%)에서 유지하여 통화정책을 운용하기로 하였음
Dovish matches: ['유지', '운용'], Count: 2
Hawkish matches: ['기준금리'], Count: 1
Predicted: Dovish

Sentence: 최근 국내 경기는 내수와 수출 모두 감소세를 지속하면서계속 위축되고 있으나 하강속도는 다소 완만해지고 있으며 세계경제 침체, 국제금융시장 불안 등으로 인해 향후성장의 하향위험은 여전히 큰 것으로 판단됨
Dovish matches: ['내수', '수출', '감소세', '지속', '위축', '하강', '완만', '해지', '침체', '국제', '성장', '하향', '위험'], Count: 13
Hawkish matches: ['불안', '성장', '위험'], Count: 3
Predicted: Dovish

Sentence: 소비자물가는 국제유가가 안정세를 보이는 가운데 경기부진에 따른 수요압력 완화로 오름세가 둔화되었으며 앞으로도 이러한 추세가 이어질 것으로 보임.
Dovish matches: ['국제', '유가', '안정세', '부진', '수요', '완화', '둔화', '추세', '보임'], Count: 9
Hawkish matches: ['압력'], Count: 1
Predicted: Dovish

Sentence: 부동산시